In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [5]:
week5_link = 'http://www.footballlocks.com/nfl_odds_week_5.shtml#Closing%20NFL%20Odds%20Week%201,%202007'

#get html text
week5_html = requests.get(week5_link).text

soup = BeautifulSoup(week5_html, 'html.parser')

#want just table row data
tr = soup.find_all("tr")

len(tr)

#start of table
print(tr[51])

#last entry in table
print(tr[270])

<tr>
<td>10/4 8:20 ET</td>
<td>At New England</td>
<td>-11</td>
<td>Indianapolis</td>
<td>50</td>
<td>-$675 +$475</td>
</tr>
<tr>
<td>10/9 8:30 ET</td>
<td>At Denver</td>
<td>-5</td>
<td>Baltimore</td>
<td>33</td>
<td>-$250 +$210</td>
</tr>


In [6]:
#want to 271 because it does not include last one
table = tr[51:271]

type(table[-1])

type(table)

string_table = [str(i) for i in table]

type(string_table[0])

week5_df = pd.DataFrame(string_table)

week5_df.columns = ['all_data']

week5_df.head()

,all_data
0,<tr>\n<td>10/4 8:20 ET</td>\n<td>At New Englan...
1,<tr>\n<td>10/7 1:00 ET</td>\n<td>Baltimore</td...
2,<tr>\n<td>10/7 1:00 ET</td>\n<td>At Kansas Cit...
3,<tr>\n<td>10/7 1:00 ET</td>\n<td>Tennessee</td...
4,<tr>\n<td>10/7 1:00 ET</td>\n<td>At Carolina</...


In [7]:
#want to clean the date data

dates = [x.split('td')[1] for x in week5_df['all_data']]

cleaned_dates = []
for x in dates:
    str1 = x.replace(">", "").replace("<", "")
    final_str = str1[:-1]
    cleaned_dates.append(final_str)

week5_df['date'] = cleaned_dates

In [9]:
#pull out favorites

week5_df['cleaned_favorites'] = week5_df['all_data'].map(lambda x: x.split('td')[3].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create underdog column

week5_df['cleaned_underdogs'] = week5_df['all_data'].map(lambda x: x.split('td')[7].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create spread column
week5_df['cleaned_spread'] = week5_df['all_data'].map(lambda x: x.split('td')[5].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create over_under column
week5_df['cleaned_over_under'] = week5_df['all_data'].map(lambda x: x.split('td')[9].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create money_odds column and clean data
money_odds = []
for x in week5_df['all_data']:
    try:
        money_odds.append(x.split('td')[11].replace('<', '').replace('>', '').replace('/', ''))
    except:
        money_odds.append(np.nan)
        
week5_df['money_odds'] = money_odds

#input week number for each game
week5_df['week_number'] = '5'

In [11]:
#want to input year for each game. on lines we want to decrease the count by one, len(week1_df.all_data[x].split('td')) = 3
year = 2018
year_list = []
for x in week5_df['all_data']:
    if len(x.split('td')) > 3:
        year_list.append(year)
    else:
        year_list.append(year)
        year -= 1

week5_df['year'] = year_list

In [12]:
#create a final dataframe for week 5
week5_final = week5_df[['date', 'year', 'week_number', 'cleaned_favorites', 'cleaned_underdogs', 'cleaned_spread', 
                        'cleaned_over_under', 'money_odds']]
week5_final.head()

,date,year,week_number,cleaned_favorites,cleaned_underdogs,cleaned_spread,cleaned_over_under,money_odds
0,10/4 8:20 ET,2018,5,At New England,Indianapolis,-11,50,-$675 +$475
1,10/7 1:00 ET,2018,5,Baltimore,At Cleveland,-3,45,-$180 +$160
2,10/7 1:00 ET,2018,5,At Kansas City,Jacksonville,-3,48,-$155 +$135
3,10/7 1:00 ET,2018,5,Tennessee,At Buffalo,-6,38,-$290 +$240
4,10/7 1:00 ET,2018,5,At Carolina,NY Giants,-6.5,43.5,-$290 +$240


In [13]:
#week5_final to pickle
week5_final.to_pickle('week5_final.pkl')